### Kütüphaneler

In [4]:
import torch
import torch.nn as nn 
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms 
import torchvision
import os
import pandas as pd
from torch.utils.data import (Dataset,DataLoader) 
from skimage import io
import time

import warnings
warnings.filterwarnings("ignore")

### Veriyi Dahil Etme

In [5]:
class veri(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        if self.transform:
            image = self.transform(image)
            
        return (image, y_label)

### Veriyi Hazırlama

In [6]:
dataset = veri(csv_file = r"f113.csv",
              root_dir = r"f1_classification",
              transform = transforms.Compose([
                  transforms.ToTensor(),
                  #transforms.ToPILImage(mode=None),
                  transforms.Resize(size = (28, 28)),
                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

### Veri Ön İşleme

In [7]:
train_set,  test_set = torch.utils.data.random_split(dataset, [200,79]) 
train_loader = DataLoader(dataset=train_set, batch_size=1, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)

### Model Mimarisini Oluşturma

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=(5,5))
        self.conv2 = nn.Conv2d(in_channels=4, out_channels=8, kernel_size=(3,3))
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(2,2)) #22
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(2,2)) #10
        
        self.max = nn.MaxPool2d(kernel_size=(2,2))
        
        self.func = nn.ELU()
        
        self.fc1 = nn.Linear(in_features=32, out_features=50)
        #self.fc1 = nn.Linear(in_features=519840, out_features=50)
        #self.fc1 = nn.Linear(in_features=32*100*100, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=50)
        self.fc3 = nn.Linear(in_features=50, out_features=100)
        self.fc4 = nn.Linear(in_features=100, out_features=4)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.func(x)
        
        x = self.max(x)
        
        x = self.conv2(x)
        x = self.func(x)
        
        x = self.max(x)
        
        x = self.conv3(x)
        x = self.func(x)
        
        x = self.max(x)
        
        x = self.conv4(x)
        x = self.func(x)
        
        #flatten
        x = x.view(x.size(0), -1)
        #x = x.view(x.size(0), 519840)
        
        x = self.fc1(x)
        x = self.func(x)
        x = self.fc2(x)
        x = self.func(x)
        x = self.fc3(x)
        x = self.func(x)
        x = self.fc4(x)
        
        return x

### Modelin Eğitimi

In [9]:
start = time.time()
model = Net()

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

error = torch.nn.CrossEntropyLoss()

from torch.optim.lr_scheduler import StepLR
# step_size kaç epoch'da bir lr'yi düşüreceğini söylüyor 1000 epochluk eğitimde
# 100 olarak verilebilir. Gamma ise ne kadar azalacağını çarpan olarak veriyor.
lr = StepLR(optimizer, step_size = 2, gamma = 0.7)


epoch = 10
count = 0
loss_list  = []
iteration_list = []
acc_list = []

for i in range(epoch):
    
    lr.step()
    print("Epoch:", i, "LR:", lr.get_lr())
    
    for images, label in train_loader:
        
        tahmin = model(images)
        
        optimizer.zero_grad()
        
        loss = error(tahmin, label)
        
        loss.backward()
        
        optimizer.step()
        
        count+=1
        
        if count % 100 == 0:
            total = 0
            correct = 0
            correct_hata = 0
            
            for images, labels in test_loader:
                out = model(images.float())
                pred = torch.max(out.data, 1)[1]
                total += len(labels)
                
                correct += (pred == labels).sum()
                correct_hata += (pred != labels).sum()
                
            dogruluk = 100 * correct / float(total)
            hata = 100 * correct_hata / float(total)
        
            loss_list.append(loss.data)
            acc_list.append(dogruluk)
            iteration_list.append(count)
            
        if count % 100 == 0:
            print(f"Iteration: {count} Loss: {loss.data} Accuracy: {dogruluk}% Error: {hata}%")
            
end = time.time()
print("Süre: ", end - start)

Epoch: 10 LR: [0.001]
Iteration: 100 Loss: 1.5472517013549805 Accuracy: 30.37974739074707% Error: 69.62025451660156%
Iteration: 200 Loss: 0.846032440662384 Accuracy: 44.30379867553711% Error: 55.69620132446289%
Epoch: 10 LR: [0.00049]
Iteration: 300 Loss: 1.0755635499954224 Accuracy: 53.16455841064453% Error: 46.83544158935547%
Iteration: 400 Loss: 0.8339922428131104 Accuracy: 48.10126495361328% Error: 51.89873504638672%
Epoch: 10 LR: [0.0007]
Iteration: 500 Loss: 1.1701064109802246 Accuracy: 63.2911376953125% Error: 36.7088623046875%
Iteration: 600 Loss: 0.21413128077983856 Accuracy: 63.2911376953125% Error: 36.7088623046875%
Epoch: 10 LR: [0.000343]
Iteration: 700 Loss: 1.2134205102920532 Accuracy: 65.82278442382812% Error: 34.177215576171875%
Iteration: 800 Loss: 0.06622816622257233 Accuracy: 65.82278442382812% Error: 34.177215576171875%
Epoch: 10 LR: [0.00049]
Iteration: 900 Loss: 1.040981411933899 Accuracy: 72.15190124511719% Error: 27.848100662231445%
Iteration: 1000 Loss: 0.0297